In [ ]:
import pandas as pd
import pyarrow.parquet as pq
parquet_file_path = '/content/part-00199-4e325fbe-f437-4b53-8a36-0ef1a9544817-c000.snappy.parquet'
if not os.path.exists(parquet_file_path):
    raise FileNotFoundError(f"File not found: {parquet_file_path}")
try:
    pq.read_table(parquet_file_path)
except Exception as e:
    raise ValueError(f"File is corrupted: {e}")
df = pd.read_parquet(parquet_file_path)
print(df)

                        full_name  \
0          Amelia Verdugo chanoca   
1                Bartomeu Gabriel   
2       danny esau herrera pineda   
3                    ester choren   
4                 Jaume balsellsj   
...                           ...   
181621         Wellington Mukunya   
181622            Willard Munjoma   
181623            William Mazviro   
181624            William Simango   
181625          Zesulwe Mnkandhla   

                                               social_url  \
0       https://ad.linkedin.com/in/amelia-verdugo-chan...   
1       https://ad.linkedin.com/in/bartomeu-gabriel-81...   
2       https://ad.linkedin.com/in/danny-esau-herrera-...   
3        https://ad.linkedin.com/in/ester-choren-079b4a17   
4       https://ad.linkedin.com/in/jaume-balsellsj-89b...   
...                                                   ...   
181621  https://zw.linkedin.com/in/wellington-mukunya-...   
181622  https://zw.linkedin.com/in/willard-munjoma-1b2...   
181623

In [ ]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227146 sha256=d9db417e715b78ca8cad8a114e2d31f593d14e0fd555763342bfe1e4a279eb18
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import pandas as pd
import fasttext
parquet_file_path = '/content/part-00199-4e325fbe-f437-4b53-8a36-0ef1a9544817-c000.snappy.parquet'
df = pd.read_parquet(parquet_file_path)
if 'model' not in globals():
    model = fasttext.load_model('lid.176.bin')

df['language'] = df['country_code'].apply(lambda x: model.predict(x, k=1)[0][0])
#output
print(df['language'])

0         __label__en
1         __label__en
2         __label__en
3         __label__en
4         __label__en
             ...     
181621    __label__en
181622    __label__en
181623    __label__en
181624    __label__en
181625    __label__en
Name: language, Length: 181626, dtype: object


In [ ]:
!pip show groq

Name: groq
Version: 0.8.0
Summary: The official Python library for the groq API
Home-page: 
Author: 
Author-email: Groq <support@groq.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 


In [ ]:
import os
import pandas as pd
from groq import Groq
import pyarrow.parquet as pq
import fasttext

%env GROQ_API_KEY=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
api_key = os.environ.get("GROQ_API_KEY")
client = Groq(api_key=api_key)
parquet_file_path = '/content/part-00199-4e325fbe-f437-4b53-8a36-0ef1a9544817-c000.snappy.parquet'
output_parquet_file_path = '/content/part-00199-4e325fbe-f437-4b53-8a36-0ef1a9544817-c000_with_language_translation.snappy.parquet'

if not os.path.exists(parquet_file_path):
    raise FileNotFoundError(f"File not found: {parquet_file_path}")
try:
    pq.read_table(parquet_file_path)
except Exception as e:
    raise ValueError(f"File is corrupted: {e}")
df = pd.read_parquet(parquet_file_path)

#client = Groq()

if 'model' not in globals():
    model = fasttext.load_model('lid.176.bin')

def detect_and_translate(text):
    if text and isinstance(text, str):
        #detected_language = language_detection_response.get('language')

        detected_language = model.predict(text, k=1)[0][0]
        if detected_language != 'en':
            translation_response = client.translate(text, target_language='en')
            translated_text = translation_response.get('translation')
        else:
            translated_text = text

        return detected_language, translated_text
    return None, None
def detect_and_translate(text):
  if text is not None:
      text = text.replace('\n', '')
      detected_language = model.predict(text, k=1)[0][0]
      if detected_language != 'en':
          translation_response = client.translate(text, target_language='en')
          translated_text = translation_response.get('translation')
      else:
          translated_text = text
      return {'detected_language': detected_language, 'translated_text': translated_text}
  return None, None

df[['detected_language', 'translated_text']] = df['about_me'].apply(lambda x: pd.Series(detect_and_translate(x)))
df.to_parquet(output_parquet_file_path, index=False)
print(df[['about_me', 'detected_language', 'translated_text']].head())



env: GROQ_API_KEY=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


AttributeError: 'Groq' object has no attribute 'translate'